In [1]:
import pandas as pd
from spark_session import LocalSparkSession
from dataset import Dataset
from mr_id3 import MapReduceIDR3
from pyspark.mllib.tree import DecisionTree

In [2]:
from log import log
from sklearn.impute import SimpleImputer
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [3]:
%time
num_fields = [
    'age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
    'hours_per_week', ]

categorical_fields = [
    'workclass', 'education',
    'marital_status', 'occupation', 'relationship',
    'race', 'sex', 'native_country', ]

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs


In [4]:
%time
target = 'label'
filename = 'dataset/adult.data'

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.77 µs


In [5]:
%time
clusters = [2, 4, ]  # list(range(1, 2))
multiply = [5, ]  # list(range(1, 2))
metrics = []

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs


In [6]:
spark = LocalSparkSession(2)
spark.start()

22/10/13 10:30:35 WARN Utils: Your hostname, Mac-Pro-de-MARCELO.local resolves to a loopback address: 127.0.0.1; using 192.168.0.62 instead (on interface en2)
22/10/13 10:30:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/13 10:30:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/13 10:30:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
dataset = Dataset(spark.spark, filename, num_fields, categorical_fields, target)
dataset.load()
dataset.string_indexer()

2022-10-13 10:30:37,876 [INFO] Dataset : Starting
2022-10-13 10:30:37,878 [INFO] Dataset : Loading Dataset /Users/marcelovasconcellos/PycharmProjects/8INF919_Devoir1_Classification-distribuee-par-arbre-de-decision/dataset/adult.data


In [8]:
1/0

ZeroDivisionError: division by zero

In [10]:
dataset.df.columns

['label',
 'age',
 'fnlwgt',
 'education_num',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'workclass_idx',
 'education_idx',
 'marital_status_idx',
 'occupation_idx',
 'relationship_idx',
 'race_idx',
 'sex_idx',
 'native_country_idx']

In [ ]:
1/0

In [15]:
df = dataset.df

In [16]:
df.show(2)

+-----+---+------+-------------+------------+------------+--------------+-------------+-------------+------------------+--------------+----------------+--------+-------+------------------+
|label|age|fnlwgt|education_num|capital_gain|capital_loss|hours_per_week|workclass_idx|education_idx|marital_status_idx|occupation_idx|relationship_idx|race_idx|sex_idx|native_country_idx|
+-----+---+------+-------------+------------+------------+--------------+-------------+-------------+------------------+--------------+----------------+--------+-------+------------------+
|    0| 39| 77516|           13|        2174|           0|            40|            4|            2|                 1|             3|               1|       0|      0|                 0|
|    0| 50| 83311|           13|           0|           0|            13|            1|            2|                 0|             2|               0|       0|      0|                 0|
+-----+---+------+-------------+------------+----------

In [17]:
from pyspark.mllib.linalg import Vectors
from pyspark.sql.types import Row

In [19]:
df.rdd.map(lambda r: Row(
        label = r.label,
#         age = r.age,
#         fnlwgt = r.fnlwgt,
#         education_num = r.education_num,
#         capital_gain = r.capital_gain,
#         capital_loss = r.capital_loss,
#         hours_per_week = r.hours_per_week,
#         workclass_idx = r.workclass_idx,
#         education_idx = r.education_idx,
#         marital_status_idx = r.marital_status_idx,
#         occupation_idx = r.occupation_idx,
#         relationship_idx = r.relationship_idx,
#         race_idx = r.race_idx,
#         sex_idx = r.sex_idx,
#         native_country_idx = r.native_country_idx,
        features = Vectors.dense(
            r.age, 
            r.fnlwgt, 
            r.education_num, 
            r.capital_gain, 
            r.capital_loss, 
            r.hours_per_week, 
            r.workclass_idx, 
            r.education_idx, 
            r.marital_status_idx, 
            r.occupation_idx, 
            r.relationship_idx, 
            r.race_idx, 
            r.sex_idx, 
            r.native_country_idx
        ))
        ).toDF().show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[39.0,77516.0,13....|
|    0|[50.0,83311.0,13....|
|    0|[38.0,215646.0,9....|
|    0|[53.0,234721.0,7....|
|    0|[28.0,338409.0,13...|
|    0|[37.0,284582.0,14...|
|    0|[49.0,160187.0,5....|
|    1|[52.0,209642.0,9....|
|    1|[31.0,45781.0,14....|
|    1|[42.0,159449.0,13...|
|    1|[37.0,280464.0,10...|
|    1|[30.0,141297.0,13...|
|    0|[23.0,122272.0,13...|
|    0|[32.0,205019.0,12...|
|    1|[40.0,121772.0,11...|
|    0|[34.0,245487.0,4....|
|    0|[25.0,176756.0,9....|
|    0|[32.0,186824.0,9....|
|    0|[38.0,28887.0,7.0...|
|    1|[43.0,292175.0,14...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
df.rdd.map(lambda r: Row(
                    label = r.label,
                    age = r.age,
                    features = Vectors.dense(r.label, r.age))
                    ).toDF().show()

In [ ]:
df.withColumn("features", Vectors.dense([df.label, df.age])) \
  .show()

In [ ]:
dataset.string_indexer()


In [ ]:
dataset.assemble_features()

In [ ]:
dataset.df.show(5)

In [ ]:
dataset.df.toPandas().head(5)

In [ ]:
1/0

In [ ]:
mr_id3 = MapReduceIDR3(dataset)

In [ ]:
mr_id3.set_labeled_point()

In [ ]:
mr_id3.split()

In [ ]:
mr_id3.dataset.df.schema

In [ ]:
mr_id3.training_data.toDF()

In [ ]:
mr_id3.train()

In [ ]:
mr_id3.model = DecisionTree.trainClassifier(
            mr_id3.training_data,
            numClasses=2,
            categoricalFeaturesInfo={},
            impurity='entropy',
            maxDepth=5,
            maxBins=42,
        )

In [ ]:
mr_id3.predictions = mr_id3.model.predict(
            mr_id3.test_data.map(lambda x: x.features))

In [ ]:
labels_and_predictions = mr_id3.test_data.map(
            lambda lp: lp.label).zip(mr_id3.predictions)

In [ ]:
dfl = labels_and_predictions.toDF()
dfl.printSchema()
dfl.show()

In [ ]:
1/0 #labels_and_predictions

In [ ]:
mr_id3.errors = labels_and_predictions.filter(
            lambda lp: lp[0] != lp[1]).count() / float(mr_id3.test_data.count())

In [ ]:
metrics = BinaryClassificationMetrics(labels_and_predictions)

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [ ]:
mr_id3.predict()

In [ ]:
mr_id3 = MapReduceIDR3(dataset)

In [ ]:
dataset.df.show(5)

In [ ]:
1/0

In [ ]:
%time
for n_clusters in clusters:
    for m_factor in multiply:
        spark = LocalSparkSession(n_clusters)
        spark.start()

        dataset = Dataset(spark.spark, filename, num_fields, categorical_fields, target)
        dataset.load()
        dataset.string_indexer()
        dataset.multiply_dataset(m_factor)

        mr_id3 = MapReduceIDR3(dataset)
        mr_id3.set_labeled_point()
        mr_id3.split()
        mr_id3.train()
        mr_id3.predict()
        
        metric = mr_id3.get_metrics()
        metric['length_rows'] = dataset.df.count()
        metric['m_factor'] = m_factor
        metric['n_clusters'] = n_clusters
        metrics.append(metric)

        spark.stop()

In [ ]:
%time
df = pd.DataFrame.from_dict(metrics)

In [ ]:
df